# Day 62: Markov Decision Processes in Reinforcement Learning

## Introduction

Welcome to Day 62 of the 100 Days of Machine Learning! Today we dive into **Markov Decision Processes (MDPs)**, a mathematical framework that forms the foundation of reinforcement learning. MDPs provide a structured way to model decision-making situations where outcomes are partly random and partly under the control of an agent.

In the previous lesson, we explored the fundamentals of reinforcement learning, including agents, environments, and reward structures. Now, we'll formalize these concepts using MDPs, which give us the mathematical tools to analyze and solve sequential decision-making problems.

### Why MDPs Matter in Machine Learning

Markov Decision Processes are crucial because they:

- Provide a mathematical framework for modeling sequential decision-making under uncertainty
- Form the theoretical foundation for most reinforcement learning algorithms
- Enable us to formalize the notion of optimal behavior in dynamic environments
- Allow us to compute optimal policies for agents in complex environments

MDPs are used in diverse applications including robotics (path planning), finance (portfolio optimization), healthcare (treatment planning), and game playing (chess, Go, video games).

### Learning Objectives

By the end of this lesson, you will be able to:

- Understand the components of a Markov Decision Process (states, actions, transitions, rewards)
- Explain the Markov property and why it's important for reinforcement learning
- Work with transition probability matrices and state-value functions
- Implement simple MDPs in Python and visualize state transitions
- Apply value iteration and policy iteration algorithms to solve MDPs
- Understand the Bellman equation and its role in computing optimal policies

## Theory: Understanding Markov Decision Processes

### What is a Markov Decision Process?

A **Markov Decision Process (MDP)** is a mathematical framework for modeling decision-making in situations where outcomes are partly random and partly under the control of a decision maker (agent). An MDP is formally defined by a tuple $(S, A, P, R, \gamma)$:

- **$S$**: A finite set of **states** representing all possible situations the agent can be in
- **$A$**: A finite set of **actions** the agent can take
- **$P$**: The **transition probability function** $P(s' | s, a)$ - the probability of transitioning to state $s'$ given we're in state $s$ and take action $a$
- **$R$**: The **reward function** $R(s, a, s')$ - the immediate reward received when transitioning from state $s$ to $s'$ via action $a$
- **$\gamma$**: The **discount factor** $\gamma \in [0, 1]$ - determines how much we value future rewards vs immediate rewards

### The Markov Property

The defining characteristic of MDPs is the **Markov property**, which states:

$$P(s_{t+1} | s_t, a_t, s_{t-1}, a_{t-1}, ..., s_0, a_0) = P(s_{t+1} | s_t, a_t)$$

In simple terms: **the future depends only on the present, not on the past**. Given the current state, the history of how we got there doesn't matter for predicting the future. This property greatly simplifies the mathematical analysis of sequential decision problems.

### Transition Probability Matrices

For a given action $a$, we can represent all transition probabilities using a **transition matrix** $P^a$, where:

$$P^a_{ij} = P(s_j | s_i, a)$$

This is the probability of transitioning from state $i$ to state $j$ when taking action $a$. Each row of the matrix must sum to 1, since from any state, we must transition somewhere (including possibly staying in the same state).

### Policies and Value Functions

A **policy** $\pi$ is a strategy that the agent follows - it maps states to actions:

$$\pi: S \rightarrow A$$

Or more generally, $\pi(a|s)$ gives the probability of taking action $a$ in state $s$ (stochastic policy).

The **state-value function** $V^\pi(s)$ represents the expected cumulative reward starting from state $s$ and following policy $\pi$:

$$V^\pi(s) = \mathbb{E}_{\pi}\left[\sum_{t=0}^{\infty} \gamma^t R_{t+1} \mid S_0 = s\right]$$

The **action-value function** (Q-function) $Q^\pi(s, a)$ represents the expected cumulative reward starting from state $s$, taking action $a$, then following policy $\pi$:

$$Q^\pi(s, a) = \mathbb{E}_{\pi}\left[\sum_{t=0}^{\infty} \gamma^t R_{t+1} \mid S_0 = s, A_0 = a\right]$$

### The Bellman Equation

The **Bellman equation** is a recursive relationship that expresses the value of a state in terms of the values of successor states:

$$V^\pi(s) = \sum_{a} \pi(a|s) \sum_{s'} P(s'|s,a) \left[R(s,a,s') + \gamma V^\pi(s')\right]$$

This equation says: the value of state $s$ equals the expected immediate reward plus the discounted value of the next state.

The **optimal value function** $V^*(s)$ satisfies the **Bellman optimality equation**:

$$V^*(s) = \max_{a} \sum_{s'} P(s'|s,a) \left[R(s,a,s') + \gamma V^*(s')\right]$$

And the optimal policy is:

$$\pi^*(s) = \arg\max_{a} \sum_{s'} P(s'|s,a) \left[R(s,a,s') + \gamma V^*(s')\right]$$

## Python Implementation

Let's implement a simple MDP and explore its properties. We'll start by importing the necessary libraries.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.patches import FancyBboxPatch, FancyArrowPatch
import pandas as pd

# Set random seed for reproducibility
np.random.seed(42)

# Set style for better-looking plots
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

### Example: GridWorld MDP

We'll create a simple GridWorld environment - a classic example in reinforcement learning. The agent moves on a grid, trying to reach a goal state while avoiding obstacles.

**GridWorld Setup:**
- 4x4 grid of states
- Agent can move: UP, DOWN, LEFT, RIGHT
- Goal state: top-right corner (reward = +10)
- Obstacle state: position (1,1) (reward = -10)
- All other moves: reward = -1 (to encourage reaching goal quickly)
- Agent stays in place if it tries to move off the grid

In [2]:
class GridWorldMDP:
    """A simple GridWorld MDP environment."""
    
    def __init__(self, size=4, goal_reward=10, obstacle_penalty=-10, step_cost=-1):
        self.size = size
        self.n_states = size * size
        self.n_actions = 4  # UP, DOWN, LEFT, RIGHT
        
        # Define actions
        self.actions = ['UP', 'DOWN', 'LEFT', 'RIGHT']
        self.action_effects = {
            0: (-1, 0),  # UP
            1: (1, 0),   # DOWN
            2: (0, -1),  # LEFT
            3: (0, 1)    # RIGHT
        }
        
        # Special states
        self.goal_state = 3  # Top-right corner (0, 3)
        self.obstacle_state = 5  # Position (1, 1)
        
        # Rewards
        self.goal_reward = goal_reward
        self.obstacle_penalty = obstacle_penalty
        self.step_cost = step_cost
        
        # Build transition matrices and reward function
        self._build_transition_matrices()
        self._build_reward_function()
        
    def _state_to_position(self, state):
        """Convert state index to (row, col) position."""
        return (state // self.size, state % self.size)
    
    def _position_to_state(self, row, col):
        """Convert (row, col) position to state index."""
        return row * self.size + col
    
    def _is_valid_position(self, row, col):
        """Check if position is within grid boundaries."""
        return 0 <= row < self.size and 0 <= col < self.size
    
    def _build_transition_matrices(self):
        """Build transition probability matrices for each action."""
        self.P = np.zeros((self.n_actions, self.n_states, self.n_states))
        
        for action in range(self.n_actions):
            for state in range(self.n_states):
                row, col = self._state_to_position(state)
                d_row, d_col = self.action_effects[action]
                
                new_row = row + d_row
                new_col = col + d_col
                
                # If move is valid, transition to new state
                if self._is_valid_position(new_row, new_col):
                    new_state = self._position_to_state(new_row, new_col)
                    self.P[action, state, new_state] = 1.0
                else:
                    # Stay in same state if move would go off grid
                    self.P[action, state, state] = 1.0
    
    def _build_reward_function(self):
        """Build reward function R(s, a, s')."""
        self.R = np.full((self.n_actions, self.n_states, self.n_states), self.step_cost)
        
        # Set rewards for reaching goal
        for action in range(self.n_actions):
            for state in range(self.n_states):
                self.R[action, state, self.goal_state] = self.goal_reward
                self.R[action, state, self.obstacle_state] = self.obstacle_penalty

# Create GridWorld MDP
env = GridWorldMDP(size=4)
print(f"GridWorld MDP created with {env.n_states} states and {env.n_actions} actions")
print(f"Goal state: {env.goal_state}, Obstacle state: {env.obstacle_state}")

GridWorld MDP created with 16 states and 4 actions
Goal state: 3, Obstacle state: 5


### Visualizing Transition Matrices

Let's visualize the transition probability matrix for the 'RIGHT' action to understand how states transition.

In [3]:
# Visualize transition matrix for 'RIGHT' action
action_idx = 3  # RIGHT

plt.figure(figsize=(10, 8))
sns.heatmap(env.P[action_idx], annot=False, cmap='Blues', cbar=True, 
            xticklabels=range(env.n_states), yticklabels=range(env.n_states))
plt.title(f'Transition Probability Matrix for Action: {env.actions[action_idx]}', fontsize=14, fontweight='bold')
plt.xlabel('Next State', fontsize=12)
plt.ylabel('Current State', fontsize=12)
plt.tight_layout()
plt.show()

# Print some example transitions
print("\nExample transitions for 'RIGHT' action:")
for state in [0, 3, 12, 15]:
    next_state = np.argmax(env.P[action_idx, state])
    row, col = env._state_to_position(state)
    next_row, next_col = env._state_to_position(next_state)
    print(f"  State {state} ({row},{col}) → State {next_state} ({next_row},{next_col})")

## Value Iteration Algorithm

**Value Iteration** is a dynamic programming algorithm that computes the optimal value function $V^*(s)$ by iteratively applying the Bellman optimality equation:

$$V_{k+1}(s) = \max_{a} \sum_{s'} P(s'|s,a) \left[R(s,a,s') + \gamma V_k(s')\right]$$

The algorithm converges to the optimal value function $V^*$, from which we can extract the optimal policy.

In [4]:
def value_iteration(env, gamma=0.9, theta=1e-6, max_iterations=1000):
    """
    Perform value iteration to find optimal value function.
    
    Parameters:
    - env: GridWorldMDP environment
    - gamma: discount factor
    - theta: convergence threshold
    - max_iterations: maximum number of iterations
    
    Returns:
    - V: optimal value function
    - policy: optimal policy
    - history: value function at each iteration (for visualization)
    """
    V = np.zeros(env.n_states)
    history = [V.copy()]
    
    for iteration in range(max_iterations):
        delta = 0
        V_new = np.zeros(env.n_states)
        
        for s in range(env.n_states):
            # Skip terminal states (goal and obstacle)
            if s == env.goal_state:
                continue
                
            # Compute value for each action
            action_values = np.zeros(env.n_actions)
            for a in range(env.n_actions):
                # Expected value for taking action a in state s
                for s_next in range(env.n_states):
                    action_values[a] += env.P[a, s, s_next] * \
                                       (env.R[a, s, s_next] + gamma * V[s_next])
            
            # Take maximum over actions
            V_new[s] = np.max(action_values)
            delta = max(delta, abs(V_new[s] - V[s]))
        
        V = V_new.copy()
        history.append(V.copy())
        
        # Check for convergence
        if delta < theta:
            print(f"Value Iteration converged in {iteration + 1} iterations")
            break
    
    # Extract optimal policy
    policy = np.zeros(env.n_states, dtype=int)
    for s in range(env.n_states):
        action_values = np.zeros(env.n_actions)
        for a in range(env.n_actions):
            for s_next in range(env.n_states):
                action_values[a] += env.P[a, s, s_next] * \
                                   (env.R[a, s, s_next] + gamma * V[s_next])
        policy[s] = np.argmax(action_values)
    
    return V, policy, history

# Run value iteration
V_optimal, policy_optimal, value_history = value_iteration(env, gamma=0.9)

print("\nOptimal State Values (reshaped as 4x4 grid):")
print(np.round(V_optimal.reshape(4, 4), 2))

print("\nOptimal Policy (as action names):")
policy_grid = np.array([env.actions[a] for a in policy_optimal]).reshape(4, 4)
print(policy_grid)

Value Iteration converged in 12 iterations

Optimal State Values (reshaped as 4x4 grid):
[[ -9.47  -7.6   -4.84  10.  ]
 [-10.61 -10.   -10.52  -5.96]
 [-11.69 -11.8  -11.38  -8.44]
 [-12.74 -12.6  -11.93 -10.14]]

Optimal Policy (as action names):
[['RIGHT' 'RIGHT' 'RIGHT' 'UP']
 ['UP' 'UP' 'UP' 'UP']
 ['UP' 'UP' 'UP' 'UP']
 ['UP' 'UP' 'UP' 'UP']]


## Visualizing the MDP Solution

Let's create visualizations to better understand the optimal policy and value function.

In [5]:
def visualize_gridworld_solution(env, V, policy):
    """Visualize the optimal value function and policy."""
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 6))
    
    # Plot 1: Value function heatmap
    V_grid = V.reshape(env.size, env.size)
    im1 = ax1.imshow(V_grid, cmap='RdYlGn', interpolation='nearest')
    ax1.set_title('Optimal State-Value Function V*(s)', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Column', fontsize=12)
    ax1.set_ylabel('Row', fontsize=12)
    
    # Add value annotations
    for i in range(env.size):
        for j in range(env.size):
            text = ax1.text(j, i, f'{V_grid[i, j]:.1f}',
                          ha="center", va="center", color="black", fontsize=10, fontweight='bold')
    
    # Mark special states
    goal_row, goal_col = env._state_to_position(env.goal_state)
    obs_row, obs_col = env._state_to_position(env.obstacle_state)
    ax1.add_patch(plt.Rectangle((goal_col-0.5, goal_row-0.5), 1, 1, 
                                fill=False, edgecolor='green', linewidth=3))
    ax1.add_patch(plt.Rectangle((obs_col-0.5, obs_row-0.5), 1, 1, 
                                fill=False, edgecolor='red', linewidth=3))
    plt.colorbar(im1, ax=ax1)
    
    # Plot 2: Policy arrows
    ax2.set_xlim(-0.5, env.size - 0.5)
    ax2.set_ylim(env.size - 0.5, -0.5)
    ax2.set_aspect('equal')
    ax2.set_title('Optimal Policy π*(s)', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Column', fontsize=12)
    ax2.set_ylabel('Row', fontsize=12)
    ax2.grid(True, alpha=0.3)
    
    # Arrow directions
    arrow_map = {
        0: (0, -0.3),   # UP
        1: (0, 0.3),    # DOWN
        2: (-0.3, 0),   # LEFT
        3: (0.3, 0)     # RIGHT
    }
    
    for state in range(env.n_states):
        row, col = env._state_to_position(state)
        action = policy[state]
        dx, dy = arrow_map[action]
        
        # Different color for goal and obstacle
        if state == env.goal_state:
            ax2.add_patch(plt.Rectangle((col-0.4, row-0.4), 0.8, 0.8, 
                                        facecolor='lightgreen', edgecolor='green', linewidth=2))
            ax2.text(col, row, 'GOAL', ha='center', va='center', 
                    fontsize=9, fontweight='bold')
        elif state == env.obstacle_state:
            ax2.add_patch(plt.Rectangle((col-0.4, row-0.4), 0.8, 0.8, 
                                        facecolor='lightcoral', edgecolor='red', linewidth=2))
            ax2.text(col, row, 'OBS', ha='center', va='center', 
                    fontsize=9, fontweight='bold')
        else:
            ax2.arrow(col - dx/2, row - dy/2, dx, dy, 
                     head_width=0.15, head_length=0.15, fc='blue', ec='blue')
    
    plt.tight_layout()
    plt.show()

visualize_gridworld_solution(env, V_optimal, policy_optimal)

### Visualizing Value Iteration Convergence

Let's see how the value function evolves during the iterations.

In [6]:
# Plot convergence of value function for selected states
plt.figure(figsize=(12, 5))

# Select a few interesting states to track
states_to_plot = [0, 3, 5, 12, 15]  # Start, Goal, Obstacle, Middle, End
state_labels = ['(0,0) Start', '(0,3) Goal', '(1,1) Obstacle', '(3,0) Bottom-left', '(3,3) Bottom-right']

for idx, state in enumerate(states_to_plot):
    values = [V[state] for V in value_history]
    plt.plot(values, marker='o', label=state_labels[idx], linewidth=2, markersize=4)

plt.xlabel('Iteration', fontsize=12)
plt.ylabel('State Value V(s)', fontsize=12)
plt.title('Convergence of Value Function During Value Iteration', fontsize=14, fontweight='bold')
plt.legend(loc='best', fontsize=10)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print(f"\nTotal iterations until convergence: {len(value_history) - 1}")

## Policy Iteration Algorithm

**Policy Iteration** is an alternative to value iteration that alternates between two steps:

1. **Policy Evaluation**: Compute the value function $V^\pi$ for the current policy $\pi$
2. **Policy Improvement**: Update the policy to be greedy with respect to the current value function

This often converges in fewer iterations than value iteration.

In [7]:
def policy_evaluation(env, policy, gamma=0.9, theta=1e-6):
    """Evaluate a policy to compute its value function."""
    V = np.zeros(env.n_states)
    
    while True:
        delta = 0
        V_new = np.zeros(env.n_states)
        
        for s in range(env.n_states):
            if s == env.goal_state:
                continue
            
            a = policy[s]
            for s_next in range(env.n_states):
                V_new[s] += env.P[a, s, s_next] * (env.R[a, s, s_next] + gamma * V[s_next])
            
            delta = max(delta, abs(V_new[s] - V[s]))
        
        V = V_new.copy()
        
        if delta < theta:
            break
    
    return V

def policy_iteration(env, gamma=0.9, max_iterations=100):
    """Perform policy iteration to find optimal policy."""
    # Start with random policy
    policy = np.random.choice(env.n_actions, size=env.n_states)
    
    for iteration in range(max_iterations):
        # Policy Evaluation
        V = policy_evaluation(env, policy, gamma)
        
        # Policy Improvement
        policy_stable = True
        new_policy = np.zeros(env.n_states, dtype=int)
        
        for s in range(env.n_states):
            old_action = policy[s]
            
            # Find best action
            action_values = np.zeros(env.n_actions)
            for a in range(env.n_actions):
                for s_next in range(env.n_states):
                    action_values[a] += env.P[a, s, s_next] * \
                                       (env.R[a, s, s_next] + gamma * V[s_next])
            
            new_policy[s] = np.argmax(action_values)
            
            if old_action != new_policy[s]:
                policy_stable = False
        
        policy = new_policy.copy()
        
        if policy_stable:
            print(f"Policy Iteration converged in {iteration + 1} iterations")
            break
    
    V = policy_evaluation(env, policy, gamma)
    return V, policy, iteration + 1

# Run policy iteration
V_pi, policy_pi, pi_iterations = policy_iteration(env, gamma=0.9)

# Compare with value iteration
print("\nComparing Value Iteration vs Policy Iteration:")
print(f"Maximum difference in value functions: {np.max(np.abs(V_optimal - V_pi)):.4f}")
print(f"Policies are identical: {np.array_equal(policy_optimal, policy_pi)}")

print("\nPolicy Iteration is often more efficient!")
print(f"  Value Iteration: {len(value_history) - 1} iterations")
print(f"  Policy Iteration: {pi_iterations} iterations")

Policy Iteration converged in 3 iterations

Comparing Value Iteration vs Policy Iteration:
Maximum difference in value functions: 0.0000
Policies are identical: True

Policy Iteration is often more efficient!
  Value Iteration: 12 iterations
  Policy Iteration: 3 iterations


## Impact of Discount Factor γ

The discount factor $\gamma$ controls how much the agent values future rewards. Let's explore its impact.

In [8]:
# Compare different discount factors
gammas = [0.5, 0.9, 0.99]
results = {}

for gamma in gammas:
    V, policy, _ = value_iteration(env, gamma=gamma, theta=1e-6)
    results[gamma] = (V, policy)

# Visualize
fig, axes = plt.subplots(1, 3, figsize=(14, 4))

for idx, gamma in enumerate(gammas):
    V, policy = results[gamma]
    V_grid = V.reshape(env.size, env.size)
    
    im = axes[idx].imshow(V_grid, cmap='RdYlGn', interpolation='nearest')
    axes[idx].set_title(f'Value Function with γ = {gamma}', fontsize=12, fontweight='bold')
    axes[idx].set_xlabel('Column')
    axes[idx].set_ylabel('Row')
    
    # Add value annotations
    for i in range(env.size):
        for j in range(env.size):
            axes[idx].text(j, i, f'{V_grid[i, j]:.1f}',
                          ha="center", va="center", color="black", fontsize=8)
    
    plt.colorbar(im, ax=axes[idx])

plt.tight_layout()
plt.show()

print("\nEffect of discount factor γ:")
for gamma in gammas:
    V, _ = results[gamma]
    print(f"\nγ = {gamma:.2f}: Average value = {V.mean():.2f}, Max value = {V.max():.2f}")
    if gamma < 0.7:
        print(f"  (Short-sighted: values future rewards at {int(gamma*100)}% per step)")
    elif gamma < 0.95:
        print(f"  (Balanced: values future rewards at {int(gamma*100)}% per step)")
    else:
        print(f"  (Far-sighted: values future rewards at {int(gamma*100)}% per step)")


Effect of discount factor γ:

γ = 0.50: Average value = -4.32, Max value = 5.00
  (Short-sighted: values future rewards at 50% per step)

γ = 0.90: Average value = -9.72, Max value = 10.00
  (Balanced: values future rewards at 90% per step)

γ = 0.99: Average value = -33.15, Max value = 10.00
  (Far-sighted: values future rewards at 99% per step)


## Hands-On Activity: Simulating Agent Behavior

Now let's simulate an agent following the optimal policy and visualize its trajectory through the grid.

In [9]:
def simulate_episode(env, policy, start_state=None, max_steps=20):
    """Simulate one episode following the given policy."""
    if start_state is None:
        # Random start (but not goal or obstacle)
        valid_starts = [s for s in range(env.n_states) 
                       if s != env.goal_state and s != env.obstacle_state]
        state = np.random.choice(valid_starts)
    else:
        state = start_state
    
    trajectory = [state]
    total_reward = 0
    
    for step in range(max_steps):
        action = policy[state]
        
        # Sample next state from transition probabilities
        next_state = np.random.choice(env.n_states, p=env.P[action, state])
        reward = env.R[action, state, next_state]
        
        total_reward += reward
        trajectory.append(next_state)
        
        # Check if reached goal or obstacle
        if next_state == env.goal_state:
            break
        if next_state == env.obstacle_state:
            break
        
        state = next_state
    
    return trajectory, total_reward

# Run several episodes
np.random.seed(42)
num_episodes = 5

for ep in range(num_episodes):
    trajectory, total_reward = simulate_episode(env, policy_optimal)
    
    print(f"Episode {ep + 1}:")
    print(f"  Start: ", end="")
    for state in trajectory:
        row, col = env._state_to_position(state)
        print(f"({row},{col})", end="")
        if state == env.goal_state:
            print(" GOAL!", end="")
        elif state == env.obstacle_state:
            print(" OBSTACLE!", end="")
        if state != trajectory[-1]:
            print(" → ", end="")
    print(f"\n  Steps: {len(trajectory) - 1}, Total reward: {total_reward}")
    print()

Episode 1:
  Start: (3,0) → (2,0) → (1,0) → (0,0) → (0,1) → (0,2) → (0,3) GOAL!
  Steps: 6, Total reward: 4.0

Episode 2:
  Start: (2,1) → (1,1) OBSTACLE!
  Steps: 1, Total reward: -11.0

Episode 3:
  Start: (3,3) → (2,3) → (1,3) → (0,3) GOAL!
  Steps: 3, Total reward: 7.0

Episode 4:
  Start: (1,2) → (0,2) → (0,3) GOAL!
  Steps: 2, Total reward: 8.0

Episode 5:
  Start: (2,2) → (1,2) → (0,2) → (0,3) GOAL!
  Steps: 3, Total reward: 7.0


## Summary: Value Iteration vs Policy Iteration

Let's create a comparison table of the two main algorithms we've explored.

In [10]:
comparison_data = {
    'Value Iteration': [
        'Updates value function directly',
        'Gradual convergence',
        'Lower (one sweep)',
        'Large state spaces',
        'Optimal (at convergence)'
    ],
    'Policy Iteration': [
        'Alternates evaluation and improvement',
        'Often fewer iterations',
        'Higher (evaluation to convergence)',
        'Small state spaces, quick convergence needed',
        'Optimal (at convergence)'
    ]
}

comparison_df = pd.DataFrame(comparison_data, 
                             index=['Algorithm Type', 'Convergence', 'Per-iteration Cost', 
                                   'When to Use', 'Policy Quality'])
comparison_df

,Value Iteration,Policy Iteration
Algorithm Type,Updates value function directly,Alternates evaluation and improvement
Convergence,Gradual convergence,Often fewer iterations
Per-iteration Cost,Lower (one sweep),Higher (evaluation to convergence)
When to Use,Large state spaces,"Small state spaces, quick convergence needed"
Policy Quality,Optimal (at convergence),Optimal (at convergence)


## Key Takeaways

Congratulations on completing Day 62! Here are the main points to remember:

- **Markov Decision Processes (MDPs)** provide a mathematical framework for sequential decision-making under uncertainty, defined by states, actions, transition probabilities, rewards, and a discount factor.

- **The Markov Property** states that the future depends only on the present state, not the history - this simplifies the analysis of sequential decision problems.

- **Transition probability matrices** encode how likely we are to move from one state to another given an action, and are fundamental to computing value functions.

- **The Bellman equation** provides a recursive relationship for computing value functions, expressing the value of a state as the immediate reward plus the discounted value of successor states.

- **Value Iteration** and **Policy Iteration** are two dynamic programming algorithms that can solve MDPs to find optimal policies - they both converge to the optimal solution but with different computational trade-offs.

- **The discount factor γ** controls how much the agent values future rewards - lower values make the agent short-sighted, while higher values make it far-sighted.

- MDPs form the theoretical foundation for reinforcement learning algorithms like Q-Learning and Deep Q-Networks (DQN), which we'll explore in upcoming lessons.

Understanding MDPs is essential for mastering reinforcement learning, as they provide the mathematical tools to formalize and solve sequential decision-making problems!

## Further Resources

To deepen your understanding of Markov Decision Processes and reinforcement learning, explore these resources:

1. **[Reinforcement Learning: An Introduction](http://incompleteideas.net/book/the-book-2nd.html)** by Sutton and Barto - The definitive textbook on reinforcement learning, with comprehensive coverage of MDPs (Chapters 3-4).

2. **[David Silver's RL Course - Lecture 2: Markov Decision Processes](https://www.youtube.com/watch?v=lfHX2hHRMVQ)** - Excellent video lecture from DeepMind's RL course covering MDP theory.

3. **[OpenAI Gym Documentation](https://gymnasium.farama.org/)** - Learn about the standard interface for RL environments, many of which are MDPs.

4. **[Reinforcement Q-Learning from Scratch in Python with OpenAI Gym](https://www.learndatasci.com/tutorials/reinforcement-q-learning-scratch-python-openai-gym/)** - Practical tutorial implementing Q-Learning (building on MDP concepts).

5. **[CS 294: Deep Reinforcement Learning](http://rail.eecs.berkeley.edu/deeprlcourse/)** - Berkeley's course with advanced RL topics building on MDP foundations.

6. **[Dynamic Programming - Wikipedia](https://en.wikipedia.org/wiki/Dynamic_programming)** - Background on the dynamic programming techniques used in value and policy iteration.

### Practice Exercises

To reinforce your learning:

- Modify the GridWorld environment to add more obstacles or change the reward structure
- Implement a larger grid (e.g., 8x8) and observe how computation time scales
- Try implementing modified policy iteration (a hybrid approach)
- Experiment with stochastic transitions (e.g., actions succeed only 80% of the time)
- Implement an MDP for a different domain (e.g., inventory management, game playing)